In [1]:
!pip install -U transformers wandb pandas matplotlib scikit-learn albumentations --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


# Import modules and load data

In [2]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [3]:
from itertools import combinations
import json
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import CLIP_model
import CLIP_data_load

2023-06-01 16:36:36.016214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Hyperparameters
train_percentage = 0.8
test_percentage = 0.2
validation_percentage = 0.15
latent_dim_imgs = 1024
latent_dim_text = 768
latent_dim_common = 512
batch_size = 128
SEED = 116
img_shape = (128,128,3)
text_input_shape = (200)
bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased
max_len = 200
model_name = 'clip_3_duplicated'

# Paths
project_location = '/root'
all_images_path = f'{project_location}/dataset/resized_train/'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
dataset_extract_location = f'{project_location}/dataset/'
model_loc = f'{project_location}/weights/{model_name}.h5'

In [8]:
!cd {project_location}

In [19]:
wandb.init(project="clip-flick",
           entity='calonca',
           config = {
              'batch_size':batch_size,
              'latent_dim_common':latent_dim_common,
              'latent_dim_text':latent_dim_text,
              'latent_dim_imgs':latent_dim_imgs,
              'img_shape':img_shape,
              'text_input_shape':text_input_shape,
              'model_name': model_name,
           })

In [9]:
# Extract images
import zipfile

with zipfile.ZipFile(zip_dataset_location) as zf:
  for member in tqdm(zf.infolist(), desc='Extracting '):
    try:
        zf.extract(member, dataset_extract_location)
    except zipfile.error as e:
        pass

Extracting :   0%|          | 0/83276 [00:00<?, ?it/s]

## **PRE TRAINED BERT MODEL**

## **PRE TRAINED FEATURE EXTRACTION NETWORK**

## **CLIP MODEL**

# Data preparation

In [10]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

concepts = concept_df.set_index('concept').T.to_dict('list')
concept_id_df.cuis = concept_id_df.cuis.apply(lambda x: x.split(';'))
concept_id_df['concepts'] = concept_id_df.cuis.apply(lambda cuis: {concepts[cui][0] for cui in cuis})

captionsDF = pd.merge(captionsDF,concept_id_df, on="ID")
captionsDF.ID = captionsDF.ID.apply(lambda x: x+'.jpg')

# **Data generation**

In [11]:
train_val, test = train_test_split(captionsDF, test_size = test_percentage, shuffle = True, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, random_state = SEED)

In [12]:
train = CLIP_data_load.concepts_to_captions_dup(train)
val = CLIP_data_load.concepts_to_captions_dup(val)

In [13]:
train.iloc[99,:]['caption']

'Vertebral column'

In [14]:
tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)

trainList = CLIP_data_load.paths_captions_emb_list(train,  all_images_path,tokenizer=tokenizer,max_len=max_len)
valList = CLIP_data_load.paths_captions_emb_list(val, all_images_path,tokenizer=tokenizer,max_len=max_len)

100%|██████████| 57720/57720 [00:19<00:00, 2893.35it/s]


In [21]:
train_gen = CLIP_data_load.ClipBaseGenerator(data = trainList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)
val_gen = CLIP_data_load.ClipBaseGenerator(data = valList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)

## **Train CLIP**

In [22]:
fen_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)

bert_model = TFDistilBertModel.from_pretrained(bert_model_name)

model = CLIP_model.get_clip_model(
    image_input_shape = img_shape,
    text_input_shape = text_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True
)

2023-06-01 16:54:03.528802: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 89.42MiB (rounded to 93763584)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-01 16:54:03.528927: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-06-01 16:54:03.528947: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 177, Chunks in use: 177. 44.2KiB allocated for chunks. 44.2KiB in use in bin. 28.0KiB client-requested in use in bin.
2023-06-01 16:54:03.528961: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 156, Chunks in use: 156. 103.5KiB allocated for chunks. 103.5KiB in use in bin. 93.2KiB client-requested in use in bin.
2023-06-01 16:54:03.528975: I tensorflow/tsl/frame

ResourceExhaustedError: Exception encountered when calling layer 'distilbert' (type TFDistilBertMainLayer).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]

Call arguments received by layer 'distilbert' (type TFDistilBertMainLayer):
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • head_mask=None
  • inputs_embeds=None
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)

In [ ]:
from wandb.keras import WandbCallback
wandb_callback = WandbCallback(monitor='loss',
                               log_batch_frequency=10,
                               save_model = False,
                               validation_steps=5)


In [ ]:
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 100, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

In [ ]:
!mkdir -p {project_location}/weights
model.save(f"{project_location}/weights/{model_name}.h5")

In [ ]:
bert_model.save_pretrained("bert")

In [ ]:
import shutil
shutil.make_archive('bert.zip', 'zip', 'bert')

'/root/bert.zip.zip'